In [283]:
import numpy as np
import pandas as pd

In [284]:
df = pd.read_csv("../Biomass_data/processed/after_eda.csv")

In [285]:
df.head()

,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,GDM_g,16.2750


## Encoding

In [286]:
df['Sampling_Date'] = pd.to_datetime(df['Sampling_Date'])

df['Sampling_Day'] = df['Sampling_Date'].dt.month * 30 + df['Sampling_Date'].dt.day

In [287]:
from sklearn.preprocessing import OneHotEncoder

In [288]:
cols_to_ohe = ['State', 'Species', 'target_name']
df_encoded = pd.get_dummies(df, columns=cols_to_ohe, drop_first=True)

In [289]:
df_encoded.head()

,sample_id,image_path,Sampling_Date,Pre_GSHH_NDVI,Height_Ave_cm,target,Sampling_Day,State_Tas,State_Vic,State_WA,...,Species_Phalaris_Ryegrass_Clover,Species_Ryegrass,Species_Ryegrass_Clover,Species_SubcloverDalkeith,Species_SubcloverLosa,Species_WhiteClover,target_name_Dry_Dead_g,target_name_Dry_Green_g,target_name_Dry_Total_g,target_name_GDM_g
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015-09-04,0.62,4.6667,0.0000,274,True,False,False,...,False,False,True,False,False,False,False,False,False,False
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015-09-04,0.62,4.6667,31.9984,274,True,False,False,...,False,False,True,False,False,False,True,False,False,False
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015-09-04,0.62,4.6667,16.2751,274,True,False,False,...,False,False,True,False,False,False,False,True,False,False
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015-09-04,0.62,4.6667,48.2735,274,True,False,False,...,False,False,True,False,False,False,False,False,True,False
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015-09-04,0.62,4.6667,16.2750,274,True,False,False,...,False,False,True,False,False,False,False,False,False,True


In [290]:
print(df_encoded.columns)

Index(['sample_id', 'image_path', 'Sampling_Date', 'Pre_GSHH_NDVI',
       'Height_Ave_cm', 'target', 'Sampling_Day', 'State_Tas', 'State_Vic',
       'State_WA', 'Species_Fescue', 'Species_Fescue_CrumbWeed',
       'Species_Lucerne', 'Species_Mixed', 'Species_Phalaris',
       'Species_Phalaris_BarleyGrass_SilverGrass_SpearGrass_Clover_Capeweed',
       'Species_Phalaris_Clover',
       'Species_Phalaris_Clover_Ryegrass_Barleygrass_Bromegrass',
       'Species_Phalaris_Ryegrass_Clover', 'Species_Ryegrass',
       'Species_Ryegrass_Clover', 'Species_SubcloverDalkeith',
       'Species_SubcloverLosa', 'Species_WhiteClover',
       'target_name_Dry_Dead_g', 'target_name_Dry_Green_g',
       'target_name_Dry_Total_g', 'target_name_GDM_g'],
      dtype='object')


## Splitting Data

In [291]:
from sklearn.model_selection import train_test_split
import numpy as np

In [292]:
unique_imgs = df['image_path'].unique()

train_imgs, test_imgs = train_test_split(
    unique_imgs,
    test_size=0.2, 
    random_state=42
    )

In [293]:
train_df = df_encoded[df['image_path'].isin(train_imgs)].reset_index(drop=True)
test_df  = df_encoded[df['image_path'].isin(test_imgs)].reset_index(drop=True)

In [294]:
exclude_cols = ['target', 'sample_id', 'image_path', 'Sampling_Date']

In [295]:
X_train = train_df.drop(columns=exclude_cols, errors='ignore')
y_train = train_df['target']
X_test  = test_df.drop(columns=exclude_cols, errors='ignore')
y_test  = test_df['target']

## XGBoost Training

In [296]:
from xgboost import XGBRegressor

In [297]:
import numpy as np
y_train_log = np.log1p(y_train)

In [298]:
xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,  
    max_depth=5,     
    subsample=0.7,       
    colsample_bytree=0.7, 
    reg_alpha=1,          
    tree_method="hist",
    n_jobs=-1,
    random_state=42
)
xgb_model.fit(X_train, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [299]:
print("XGBoost model training...")
xgb_model.fit(X_train, y_train_log)

XGBoost model training...


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [300]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, GroupKFold

In [301]:
def evaluate_model_cv (scores):
    print("---------------------EVALUATION----------------------")
    print(f"Fit Time: {np.mean(scores['fit_time'])}")
    print(f"Score Time: {np.mean(scores['score_time'])}")
    print(f"RMSE:{-np.mean(scores['test_neg_root_mean_squared_error'])}")
    print(f"MAE:{-np.mean(scores['test_neg_mean_absolute_error'])}")
    print(f"MSE:{-np.mean(scores['test_neg_mean_squared_error'])}")
    print(f"R-squared (R2):{np.mean(scores['test_r2'])}")

In [302]:
groups = train_df['image_path']
gkf = GroupKFold(n_splits=5)

scoring = {
    'neg_root_mean_squared_error': 'neg_root_mean_squared_error',
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'r2': 'r2'
}

scores = cross_validate(
    xgb_model,
    X_train,
    y_train,
    cv=gkf,
    groups=groups,
    scoring=scoring,
    return_train_score=False
)

evaluate_model_cv(scores)

---------------------EVALUATION----------------------
Fit Time: 0.3006418228149414
Score Time: 0.0060332298278808595
RMSE:11.230139789181624
MAE:7.7811871914608535
MSE:128.2873094100635
R-squared (R2):0.8039446746294916


In [303]:
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print("---------------------TEST RESULTS----------------------")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE:  {mae:.4f}")
    print(f"R2:   {r2:.4f}")

In [304]:
y_pred_log = xgb_model.predict(X_test)
y_pred = np.expm1(y_pred_log)

evaluate_model(y_test, y_pred)

---------------------TEST RESULTS----------------------
RMSE: 12.6091
MAE:  7.2437
R2:   0.7894


In [305]:
from sklearn.metrics import r2_score
def evaluate_detailed(model, X_train, y_train, X_test, y_test, train_names, test_names):
    y_train_pred_log = model.predict(X_train)
    y_test_pred_log = model.predict(X_test)

    y_train_pred = np.expm1(y_train_pred_log)
    y_test_pred = np.expm1(y_test_pred_log)

    train_res = X_train.copy()
    train_res['Actual'] = y_train  
    train_res['Predicted'] = y_train_pred
    train_res['target_name'] = train_names

    test_res = X_test.copy()
    test_res['Actual'] = y_test  
    test_res['Predicted'] = y_test_pred
    test_res['target_name'] = test_names

    print(f"{'Target Name':<20} | {'Train R2':<10} | {'Test R2':<10} | {'Gap':<10}")
    print("-" * 60)
    
    unique_targets = test_res['target_name'].unique()

    for target in unique_targets:
        tr_subset = train_res[train_res['target_name'] == target]
        te_subset = test_res[test_res['target_name'] == target]

        r2_train = r2_score(tr_subset['Actual'], tr_subset['Predicted'])
        r2_test  = r2_score(te_subset['Actual'], te_subset['Predicted'])
        gap = r2_train - r2_test

        print(f"{target:<20} | {r2_train:.4f}     | {r2_test:.4f}     | {gap:.4f}")

In [306]:
train_names = df[df['image_path'].isin(train_imgs)]['target_name'].values
test_names  = df[df['image_path'].isin(test_imgs)]['target_name'].values

evaluate_detailed(xgb_model, X_train, y_train, X_test, y_test, train_names, test_names)

Target Name          | Train R2   | Test R2    | Gap       
------------------------------------------------------------
Dry_Clover_g         | 0.3057     | 0.3310     | -0.0253
Dry_Dead_g           | 0.4155     | 0.3030     | 0.1124
Dry_Green_g          | 0.7872     | 0.6827     | 0.1045
Dry_Total_g          | 0.8054     | 0.7467     | 0.0587
GDM_g                | 0.8511     | 0.8165     | 0.0346
